In [119]:
# %%
from pprint import pprint
import pandas as pd
from dataclasses import dataclass
from datetime import date, time
from IPython.display import Markdown

## EmpList

In [102]:
class EmployeeList:
   employees: list
   count: int
   
   def __init__(self) -> None:
      self.employees = []
      self.count = len(self.employees)
      
   def __repr__(self) -> str:
      display = ""
      display += str(self.count)
      display += "\n"
      display += str(self.employees)
      return display
   
   def getTrainedForShift (self,shift) -> list:
      trained = []
      for x in self.employees:
         if shift in x.shiftsCanWork:
            trained.append(x)
      return len(trained), trained
   
   def trainingBreakdown (self):
      shifts = "MI 7C 7P S OP EI EP 3 N".split(" ")
      dic = {}
      for x in shifts:
         dic.update({x: self.getTrainedForShift(x)[1]})
      return dic
   
   def __addEmployee__ (self,Employee):
      self.employees.append(Employee)
      self.count = len(self.employees)
      return str(self.count) + " employees in list"



## Shift Class

In [133]:
class Shift:
   name:                str
   time:                time
   hours:               float
   days_on:             list
   
   def __init__(self,name,time,hours,days_on):
      self.name = name
      self.time = time
      self.hours = hours
      self.days_on = days_on
      
   
   def __repr__(self) -> str:
      n = self.name; t = self.time ; hr = self.hours; dyon = self.days_on
      return f'[{n}]'
   
   @property
   def outputList (self) -> list:
      n = self.name; t = self.time ; hr = self.hours; dyon = self.days_on
      return [n, t, hr, dyon]

In [153]:
MI = Shift  ("MI",   time(6,30), 10,   [0,1,2,3,4,5,6])
_7C = Shift ("7C",   time(7),    10,   [0,1,2,3,4,5,6])
_7P = Shift ("7P",   time(7),    10,   [0,1,2,3,4,5,6])
S = Shift   ("S",    time(8),    10,   [1,2,3,4,5])
OP = Shift  ("OP",   time(8),    8,    [1,2,3,4,5])
EI = Shift  ("EI",   time(12,30),10,   [0,1,2,3,4,5,6])
EP = Shift  ("EP",   time(12,30),10,   [0,1,2,3,4,5,6])
_3 = Shift  ("3",    time(15),   10,   [0,1,2,3,4,5,6])

techShifts = [MI,_7C,_7P,S,OP,EI,EP,_3]
data = [x.outputList for x in techShifts]
df_shift = pd.DataFrame(data,columns=['name','time','length','days'])

def getTimeOfDay (time):
   if time < time(12):
      return 'AM'
   elif time < time(17):
      return 'PM'
   else:
      return 'NT'

df_shift

,name,time,length,days
0,MI,06:30:00,10,"[0, 1, 2, 3, 4, 5, 6]"
1,7C,07:00:00,10,"[0, 1, 2, 3, 4, 5, 6]"
2,7P,07:00:00,10,"[0, 1, 2, 3, 4, 5, 6]"
3,S,08:00:00,10,"[1, 2, 3, 4, 5]"
4,OP,08:00:00,8,"[1, 2, 3, 4, 5]"
5,EI,12:30:00,10,"[0, 1, 2, 3, 4, 5, 6]"
6,EP,12:30:00,10,"[0, 1, 2, 3, 4, 5, 6]"
7,3,15:00:00,10,"[0, 1, 2, 3, 4, 5, 6]"


In [141]:
class Employee:
   name:                str
   shiftsCanWork :      list
   countShiftsCanWork : int
   weekend:             int
   
   def __init__(self, name:str, shiftsCanWork:list, weekend:int, fte:float) -> None:
      """---EMPLOYEE-------------------------
         >>> Name
         >>> Shifts
         >>> Weekend#
         >>> FTE Fraction of 1
      ~-------------------------------------------------
      """
      self.name = name
      self.shiftsCanWork = shiftsCanWork
      self.shifts = [x.name for x in self.shiftsCanWork]
      self.weekend = weekend
      self.fte = fte
      self.countShiftsCanWork = len(self.shiftsCanWork)
      
   def __repr__(self) -> str:
      n = self.name ; we = self.weekend; fte = self.fte ; scw = self.countShiftsCanWork
      return f'{n} Weekend {we} | {fte}FTE | {scw} Trained Shifts'
   
   @property
   def outputList (self) -> list:
      return [self.name, self.shifts, self.fte, self.weekend]


In [98]:
Josh = Employee ("Josh", techShifts, 1, 0.625)
Brittanie = Employee ("Brittanie", [EP,_3,OP], 1, 1)
Thelassa = Employee ("Thelassa", [OP], 0, 1)
Brianna = Employee ("Brianna",[_7P],2,1)
Trisha = Employee ("Trisha", [_7P],1,1)
Michael = Employee ("Michael",techShifts, 3, 1)
Esperanza = Employee ("Esperanza",techShifts, 2, 1)
CPHTS = [Josh, Brittanie, Thelassa, Brianna, Michael, Esperanza, Trisha]
data = [x.outputList for x in CPHTS]
df = pd.DataFrame(data, columns = ["name","shifts","fte","weekendID"])
df


,name,shifts,fte,weekendID
0,Josh,"[MI, 7C, 7P, S, OP, EI, EP, 3]",0.625,1
1,Brittanie,"[EP, 3, OP]",1.000,1
2,Thelassa,[OP],1.000,0
3,Brianna,[7P],1.000,2
4,Michael,"[MI, 7C, 7P, S, OP, EI, EP, 3]",1.000,3
5,Esperanza,"[MI, 7C, 7P, S, OP, EI, EP, 3]",1.000,2
6,Trisha,[7P],1.000,1


In [99]:
df.explode("shifts").set_index("shifts").groupby('shifts').agg(lambda x: list(x))

,name,fte,weekendID
shifts,,,
3,"[Josh, Brittanie, Michael, Esperanza]","[0.625, 1.0, 1.0, 1.0]","[1, 1, 3, 2]"
7C,"[Josh, Michael, Esperanza]","[0.625, 1.0, 1.0]","[1, 3, 2]"
7P,"[Josh, Brianna, Michael, Esperanza, Trisha]","[0.625, 1.0, 1.0, 1.0, 1.0]","[1, 2, 3, 2, 1]"
EI,"[Josh, Michael, Esperanza]","[0.625, 1.0, 1.0]","[1, 3, 2]"
EP,"[Josh, Brittanie, Michael, Esperanza]","[0.625, 1.0, 1.0, 1.0]","[1, 1, 3, 2]"
MI,"[Josh, Michael, Esperanza]","[0.625, 1.0, 1.0]","[1, 3, 2]"
OP,"[Josh, Brittanie, Thelassa, Michael, Esperanza]","[0.625, 1.0, 1.0, 1.0, 1.0]","[1, 1, 0, 3, 2]"
S,"[Josh, Michael, Esperanza]","[0.625, 1.0, 1.0]","[1, 3, 2]"


In [100]:
df.groupby('weekendID').agg(lambda x : list(x))

,name,shifts,fte
weekendID,,,
0,[Thelassa],[[OP]],[1.0]
1,"[Josh, Brittanie, Trisha]","[[MI, 7C, 7P, S, OP, EI, EP, 3], [EP, 3, OP], ...","[0.625, 1.0, 1.0]"
2,"[Brianna, Esperanza]","[[7P], [MI, 7C, 7P, S, OP, EI, EP, 3]]","[1.0, 1.0]"
3,[Michael],"[[MI, 7C, 7P, S, OP, EI, EP, 3]]",[1.0]


In [120]:
Markdown(df.to_markdown())

|    | name      | shifts                                         |   fte |   weekendID |
|---:|:----------|:-----------------------------------------------|------:|------------:|
|  0 | Josh      | ['MI', '7C', '7P', 'S', 'OP', 'EI', 'EP', '3'] | 0.625 |           1 |
|  1 | Brittanie | ['EP', '3', 'OP']                              | 1     |           1 |
|  2 | Thelassa  | ['OP']                                         | 1     |           0 |
|  3 | Brianna   | ['7P']                                         | 1     |           2 |
|  4 | Michael   | ['MI', '7C', '7P', 'S', 'OP', 'EI', 'EP', '3'] | 1     |           3 |
|  5 | Esperanza | ['MI', '7C', '7P', 'S', 'OP', 'EI', 'EP', '3'] | 1     |           2 |
|  6 | Trisha    | ['7P']                                         | 1     |           1 |

In [106]:
class Schedule:
   startDate : str
   length : int
   
   def __init__(self, start, length, empls:EmployeeList) -> None:
      self.startDate = start
      self.length = length
      self.employees = empls
   def weekendsOff (self):
      suns = list(range(0,self.length,7))
      sats = list(range(6,self.length,7))
      w0 = suns + sats                 ; w0.sort()
      w1 = suns[0::2] + sats[1::2]     ; w1.sort()
      w2 = suns[1::2] + sats[0::2]     ; w2.sort()
      
      group = []

# %%


In [109]:
S1 = Schedule(date(2022,5,6),42,CPHTS)
S1.employees

[Josh Weekend 1 | 0.625FTE | 8 Trained Shifts,
 Brittanie Weekend 1 | 1FTE | 3 Trained Shifts,
 Thelassa Weekend 0 | 1FTE | 1 Trained Shifts,
 Brianna Weekend 2 | 1FTE | 1 Trained Shifts,
 Michael Weekend 3 | 1FTE | 8 Trained Shifts,
 Esperanza Weekend 2 | 1FTE | 8 Trained Shifts,
 Trisha Weekend 1 | 1FTE | 1 Trained Shifts]